<h1>Getting engagement data for tweets Twitter Climate Change sentiment dataset</h1>

In [35]:
import pandas as pd
import numpy as np
import tweepy 
import os

<h2>Load Data</h2>

In [2]:
tweets = pd.read_csv("../twitter_sentiment_data.csv")

In [3]:
tweets.head()

,sentiment,message,tweetid
0,-1,@tiniebeany climate change is an interesting h...,792927353886371840
1,1,RT @NatGeoChannel: Watch #BeforeTheFlood right...,793124211518832641
2,1,Fabulous! Leonardo #DiCaprio's film on #climat...,793124402388832256
3,1,RT @Mick_Fanning: Just watched this amazing do...,793124635873275904
4,2,"RT @cnalive: Pranita Biswasi, a Lutheran from ...",793125156185137153


<h2>Extract Engagement</h2>

In [39]:
client = tweepy.Client(bearer_token)



In [5]:
def extract_entities(t):
    keys = list(t.data.entities.keys())
    if 'mentions' in keys:
        mentioned_users = [i['username'] for i in t.data.entities['mentions']]
    else:
        mentioned_users = None
    if 'hashtags' in keys:
        hashtags = [i['tag'] for i in t.data.entities['hashtags']]
    else: 
        hashtags = None
    if 'annnotations' in keys:
        annotations = [i['annotation'] for i in t.data.entities['annotations']]
    else: 
        annotations = None
    return (mentioned_users, hashtags, annotations)
        

In [26]:
def get_tweet_data(t):
    t = client.get_tweet(t, tweet_fields=['entities', 'public_metrics', 'author_id'])
    if t.data == None:
        return []
    #identifiers
    tweet_id = t.data.id
    tweeter_id = t.data.author_id
    #get tweet attributes
    if getattr(t.data, 'public_metrics') != None:
        metrics = t.data.public_metrics
        retweets = metrics['retweet_count']
        replies = metrics['reply_count']
        likes = metrics['like_count']
        quote_tweets = metrics['quote_count']
    else:
        retweets = None
        replies = None
        likes = None
        quote_tweets = None
    #get tweet entities
    if getattr(t.data, 'entities'):
        mentioned_users, hashtags, annotations = extract_entities(t)
    else:
        mentioned_users, hashtags, annotations = None, None, None
    data = np.array([tweet_id, tweeter_id, retweets, replies, likes, quote_tweets, mentioned_users,
                     hashtags, annotations], dtype=object)
    return data

In [27]:
ids = tweets['tweetid'].values[:100]
engagement_data = [get_tweet_data(i) for i in ids]

<h2>Write to file</h2>

In [29]:
df = pd.DataFrame(data=engagement_data, columns = ['tweet_id', 'tweeter_id', 'retweets', 
                                                   'replies', 'likes', 'quote_tweets', 
                                                   'mentioned_users', 'hashtags', 'annotations']
                 )

In [37]:
os.chdir("../")

In [34]:
df.to_csv("engagement_data.csv")

,tweet_id,tweeter_id,retweets,replies,likes,quote_tweets,mentioned_users,hashtags,annotations
24,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
40,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
25,7.931366e+17,7.455940e+17,0.0,0.0,0.0,0.0,None,None,None
21,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
36,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
